Homework 3  
Assignment Group 12  
Ke Xu  
Fangzhou Liu

## 1

we are using pyspellchecker to do the test.  
To install pyspellchecker:  
pip install pyspellchecker 

In [1]:
from spellchecker import SpellChecker
spell = SpellChecker()

# find those words that may be misspelled
misspelled = spell.unknown(['aadresa','auxillary','adres'])

for word in misspelled:
    # Get the one `most likely` answer
    print(spell.correction(word))

auxiliary
address
acres


For this spell checker, it has good dictionary so it has less unknow words. And it can deal with the word which has edit distance 3. Any other suggestions like using N-gram model and improve the error model(like change 'adres' to 'address') doesn't implement in this spell checker.In my opinion, his suggestions are really difficult to implement, I can't find a spell cheker is able to implement two of his suggestions.  

## 2

In [3]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import names
import re
def word_feats(words):
    return dict([(word, True) for word in words])
 
comedy = [ 'fun', 'couple', 'love', 'love', 'couple', 'fly', 'fast', 'fun', 'fun' ]
action = [ 'fast', 'furious','shoot', 'furious', 'shoot', 'shoot', 'fun', 'fly', 'fast', 'shoot', 'love' ]

 
comedy_features= [(word_feats(com), 'com') for com in comedy]
action_features = [(word_feats(act), 'act') for act in action]
 
train_set = comedy_features + action_features
 
classifier = NaiveBayesClassifier.train(train_set) 

# Predict
com = 0
act = 0
s = "fast, couple, shoot, fly"
s = s.lower()
s = re.sub(r'[^\w\s]','',s)
words = s.split(' ')
for word in words:
    classResult = classifier.classify( word_feats(word))
    if classResult == 'com':
        com = com + 1
    if classResult == 'act':
        act = act + 1
 
print('comedy: ' + str(float(com)/len(words)))
print('action: ' + str(float(act)/len(words)))
print('we cant decide which class this review should belong to.')

comedy: 0.5
action: 0.5
we cant decide which class this review should belong to.


## 3

The basic idea of this program: we divide the training data in two part, comedy and action. Then we mark every word in comedy class with 'com' and mark every word in action class with 'act, and build a classifier. Next, we input our test data, compare every word between test data and classifier. If any word more likely in comedy class then count 1 for comedy class, and if any word more likely in action class then count 1 in action class. Finally, we compute the probability of each class, find which class has highest probability, then set this review to that class. That how it works.  
Back to problem 2, our test review is "fast, couple, shoot, fly" this cause both comedy and action class count 2, so we can't decide which class this review should belong to.  
SentiWordNet is a lexical resource in which each WordNet synset is associated to three numerical scores Obj(s), Pos(s) and Neg(s), describing how objective, positive, and negative the terms contained in the synset are. Then input a review, we compare every word between the WordNet and review, and find how many postive, negative and objective words. And then compute the probablity of each of them. Then find the highest probablity, and set this review to that class(positive, negative, objective). 


## 4

In [4]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
import re

# data for multinomial Naive Bayes count frequency for the words [good, poor, great]
X = np.array([[3,0,3],[0,1,2],[1,3,0],[1,5,2],[0,2,0]])

# data for Binarized naive Bayes only count occurrence [good, poor, great]
X_bin = np.array([[1,0,1],[0,1,1],[1,1,0],[1,1,1],[0,1,0]])

y= np.array(['pos','pos','neg','neg','neg'])
clf = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
clf2 = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
clf.fit(X, y)
clf2.fit(X_bin, y)

#predict
s ="A good, good plot and great characters but poor acting."
s = re.sub(r'[^\w\s]','',s)
s = s.lower()
good = 0
poor = 0
great = 0
words = s.split(' ')
for i in words:
    if i == 'good':
        good = good +1
    if i == 'poor':
        poor = poor +1
    if i == 'great':
        great = great +1
test = np.array([[good, poor, great]])

# test set for Binarized naive Bayes
if good > 0: 
    good1 = 1
else:
    good1 = good
if poor > 0: 
    poor1 = 1
else:
    poor1 = poor
if great > 0: 
    great1 = 1
else:
    great1 = great
test1 = np.array([[good1, poor1, great1]])
print('The multinomial Naive Bayes:')
print(clf.predict(test))
print('Binarized naive Bayes:')
print(clf2.predict(test1))

The multinomial Naive Bayes:
['pos']
Binarized naive Bayes:
['neg']


We use both multionomial naive bayes and binarized naive bayes to do this problem. And we found the results are different. For the nultionomial naive bayes the result is positive but the binarized naive bayes the result is negative.  
For multionomial naive bayes, this model is depend on frequency. Since our test review has 2 'good', 1 'poor' and 1 'great', this model will more likely to set this review to positive class depend on our training data.  
For binarized naive bayes, it only count occurrence, frequency doesn't affect this model. From our training set, we found that when 'poor' appears in a review then this review will be negative. And the test set has 1 poor. Thus, the model set this test review to negative class.  
That is the reason why we got two different results.